In [2]:
import pandas as pd
import numpy as np

In [3]:
def process_seasontickets(seasontickets_path, chair_mapping_path, sf_path):
    # Load all_tickets CSV
    seasontickets = pd.read_csv(seasontickets_path)

    # Rename columns
    seasontickets = seasontickets.rename(columns={
        'Product Id': 'Product ID',
        'Product': 'Asset Name',
        'Id': 'Roboticket ID',
        'Price area': 'Price Area Name',
        'Ticket price types': 'Price Name',
        'Base price': 'Base Price',
        'Area': 'Sector',
        'Type': 'Operation Name',
        'Date.1': 'Product Date',
        'User Id': 'Owner User ID',
        'assign using  ID number': 'Owner Identifier'
    })

    # Add missing columns
    new_columns = [
        'Season ID', 'From', 'To', 'Stadium ID', 'Product Type', 'Product Name', 'Operation', 'Return Type', 'Return ID',
        'Platform', 'Entrance Text', 'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner',
        'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Purchaser Account',
        'Return Asset', 'Status', 'Is Ticket', 'Is Season Ticket', 'Created By', 'Last Modified By'
    ]
    for col in new_columns:
        if col not in seasontickets.columns:
            seasontickets[col] = None

    # Reorder columns
    desired_columns = [
        'Product ID', 'Season ID', 'Asset Name', 'Roboticket ID', 'Price', 'Price Name', 'Price Area Name', 'Base Price', 'From', 'To',
        'Stadium ID', 'Sector', 'Row', 'Number', 'Product Type', 'Operation Name', 'Product Name', 'Product Date',
        'Operation', 'Return Type', 'Return ID', 'Owner User ID', 'Owner Identifier', 'Platform', 'Entrance Text',
        'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner', 'Purchaser Account', 'Stadium',
        'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Transaction', 'Return Asset', 'Status', 'Is Ticket',
        'Is Season Ticket', 'Created By', 'Last Modified By'
    ]
    for col in desired_columns:
        if col not in seasontickets.columns:
            seasontickets[col] = None
    seasontickets = seasontickets[desired_columns]

    # Set default values
    seasontickets['Season ID'] = 24
    seasontickets['Product Name'] = seasontickets['Asset Name']
    seasontickets['Product Type'] = 'Match'
    seasontickets['Operation Name'] = 'Sale'
    seasontickets['Purchaser Account'] = seasontickets['Owner User ID']
    seasontickets['Is Ticket'] = False
    seasontickets['Is Season Ticket'] = True
    seasontickets['Stadium ID'] = np.where(
        seasontickets['Stadium'] == 'פיס ארנה', 1, seasontickets['Stadium ID']
    )

    # Map Operation Name to Operation Code
    conditions = [
        seasontickets['Operation Name'] == 'Sale',
        seasontickets['Operation Name'] == 'Return'
    ]
    choices = [1, 2]
    seasontickets['Operation'] = np.select(conditions, choices, default=seasontickets['Operation'])

    # Define mapping function for Hebrew sectors
    def map_sector_to_section_name(sector):
        sector = str(sector).strip()
        if sector.startswith('אולם'):
            number = ''.join(filter(str.isdigit, sector))
            return f'Court_{number}' if number else None
        elif 'תא צפוני' in sector:
            return 'Suite_21'
        elif 'תא צפיה' in sector:
            number = ''.join(filter(str.isdigit, sector))
            if number and 1 <= int(number) <= 16:
                return f'Suite_{number}'
        elif 'פרקט דרומי' in sector:
            return 'Court_side_17'
        elif 'פרקט צפוני' in sector:
            return 'Court_side_19'
        elif 'פרקט מרכזי' in sector or 'פרקט מרכז' in sector:
            return 'Court_side_18'
        elif 'גלריה' in sector:
            number = ''.join(filter(str.isdigit, sector))
            return f'Gallery_{number}' if number else None
        elif 'Courtyard' in sector:
            return 'Unmarked_12'
        return None

    # Map section name
    seasontickets['section_name'] = seasontickets['Sector'].apply(map_sector_to_section_name)
    seasontickets['Row'] = seasontickets['Row'].astype(str).str.strip()
    seasontickets['Number'] = seasontickets['Number'].astype(str).str.strip()

    # Load chair mapping
    chair_mapping = pd.read_excel(chair_mapping_path)
    chair_mapping['row_number'] = chair_mapping['row_number'].astype(str).str.strip()
    chair_mapping['seat_number'] = chair_mapping['seat_number'].astype(str).str.strip()

    # Merge to get Seat ID
    seasontickets = seasontickets.merge(
        chair_mapping[['seat_id', 'section_name', 'row_number', 'seat_number']],
        how='left',
        left_on=['section_name', 'Row', 'Number'],
        right_on=['section_name', 'row_number', 'seat_number']
    )

    seasontickets['Seat ID'] = seasontickets['seat_id']
    seasontickets.drop(columns=['seat_id', 'section_name', 'row_number', 'seat_number'], inplace=True)

    # Load SF data
    sf = pd.read_csv(sf_path)
    seasontickets_ids = seasontickets[['Owner Identifier', 'Account ID']].copy()
    merged_ids = seasontickets_ids.merge(
        sf[['id__c', 'AccountId']].astype(str).apply(lambda x: x.str.strip()),
        how='left',
        left_on='Owner Identifier',
        right_on='id__c'
    )
    merged_ids['Account ID'] = merged_ids['Account ID'].fillna(merged_ids['AccountId'])
    merged_ids.drop(columns=['id__c', 'AccountId'], inplace=True)
    seasontickets['Account ID'] = merged_ids['Account ID']

    return seasontickets


In [7]:
df = process_seasontickets(
    seasontickets_path='seasontickets.csv',
    chair_mapping_path='chair_mapping_20240801.xlsx',
    sf_path='extract.csv'
)

In [8]:
df.iloc[90]

Product ID                                62
Season ID                                 24
Asset Name              מנוי לעונת 2024/2025
Roboticket ID                         872742
Price                                    350
Price Name                             Adult
Price Area Name                            D
Base Price                               700
From                                    None
To                                      None
Stadium ID                              None
Sector                                אולם 6
Row                                        8
Number                                     4
Product Type                           Match
Operation Name                          Sale
Product Name            מנוי לעונת 2024/2025
Product Date                      2025-02-11
Operation                                  1
Return Type                             None
Return ID                               None
Owner User ID                          32200
Owner Iden

In [9]:
df.to_excel('seasontickets2425.xlsx')